In [1]:
!pip install gridstatusio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.1/931.1 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 9.5 MB/s eta 0:00:00
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4832 sha256=1da683bd1787cc73ff2ea4a976db41e12b56ff26e8fb0128b4d21528b7650144
  Stored in directory: /root/.cache/pip/wheels/9f/ca/d7/8f2399d70f5d641d4fef951a78d6aa4ab64cc582e1c4dec9a3
Successfully built termcolor
  Attempting uninstall: termcolor
    Found existing installation: termcolor 3.1.0
    Uninstalling termcolor-3.1.0:
      Successfully uninstalled termcolor-3.1.0
  Attempting uninstall: set

In [1]:
%pylab inline
import pandas as pd
from gridstatusio import GridStatusClient
import os

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [5]:
api_key = ''
print(api_key)

2d06f27c179d4fcebff75d36c37e0aa0


In [2]:
## mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
## read in API key and define key variables
# from google.colab import userdata
# api_key = userdata.get("GRIDSTATUS_API_KEY")

# Set start and end dates for summertime data
year = '2023'
start_date = f"{year}-06-01"
end_date = f"{year}-09-30"

In [39]:
client  = GridStatusClient(api_key=api_key)
limit_rows = 9_000 # limit of rows for hourly data for one year
#limit_rows = 100

# 1) Hourly demand
df_load = client.get_dataset(
    "ercot_standardized_hourly",
    start=start_date,
    end=end_date,
    limit=limit_rows
)
#df_load = df_load[df_load.parameter == "Demand"]
df_load["timestamp"] = pd.to_datetime(df_load["interval_start_utc"].dt.tz_convert("US/Central"))

# 2) Hourly day‐ahead hub price
df_price = client.get_dataset(
    "ercot_spp_day_ahead_hourly",
    start=start_date,
    end=end_date,
    filter_column="location",
    filter_value="HB_HUBAVG",
    limit=limit_rows
)
df_price["timestamp"] = pd.to_datetime(df_price["interval_start_utc"]).dt.tz_convert("US/Central")

# 3) Merge on the timestamp
df = pd.merge(
    df_load[["timestamp", "load.load", "fuel_mix.natural_gas", "fuel_mix.solar", "fuel_mix.power_storage", "fuel_mix.wind", "net_load"]],
    df_price[["timestamp", "spp"]],
    on="timestamp",
    suffixes=("_demand", "_da_price")
)

# 4) Save or continue your analysis
df.to_csv(f"ercot_hourly_demand_and_price_{year}.csv", index = False)
print(f"Saved merged hourly demand + DA price for {year} summer.")

Fetching Page 1...Done in 0.26 seconds. 
Total rows: 0/9,000 (0.0% of limit)

Total number of rows: 0
Fetching Page 1...Done in 0.28 seconds. 
Total rows: 0/9,000 (0.0% of limit)

Total number of rows: 0
Saved merged hourly demand + DA price for 2010 summer.


### read battery storage

only 2024 has summertime battery charge/discharge data.  presumably b/c there was so little storage in 2023.

In [ ]:
limit_rows = 150*288 # limit of rows for hourly data for 4 months

df_battery = client.get_dataset(
    "ercot_energy_storage_resources",
    start=start_date,
    end=end_date,
    limit=limit_rows
)
df_battery["timestamp"] = pd.to_datetime(df_battery["time_utc"]).dt.tz_convert("US/Central")
#df_battery = df_battery[["timestamp", "totalCharging", "totalDischarging", "netOutput"]]

Fetching Page 1...Done in 0.88 seconds. 
Total rows: 34,829/43,200 (80.62% of limit)

Total number of rows: 34829


In [ ]:
df_battery

,time_utc,total_charging,total_discharging,net_output,timestamp
0,2024-06-01 00:00:00+00:00,-121.912,461.572,339.661,2024-05-31 19:00:00-05:00
1,2024-06-01 00:05:00+00:00,-195.664,293.461,97.797,2024-05-31 19:05:00-05:00
2,2024-06-01 00:10:00+00:00,-111.288,180.916,69.627,2024-05-31 19:10:00-05:00
3,2024-06-01 00:15:00+00:00,-32.751,203.710,170.958,2024-05-31 19:15:00-05:00
4,2024-06-01 00:20:00+00:00,-47.858,173.270,125.412,2024-05-31 19:20:00-05:00
...,...,...,...,...,...
34824,2024-09-29 23:35:00+00:00,-73.902,2437.750,2363.847,2024-09-29 18:35:00-05:00
34825,2024-09-29 23:40:00+00:00,-58.184,2619.422,2561.238,2024-09-29 18:40:00-05:00
34826,2024-09-29 23:45:00+00:00,-157.681,2447.893,2290.212,2024-09-29 18:45:00-05:00
34827,2024-09-29 23:50:00+00:00,-281.372,2484.050,2202.678,2024-09-29 18:50:00-05:00


In [ ]:
# prompt: Using dataframe df_battery: convert timestamp to index, drop time_utc column, average the data over an hour
# Convert 'timestamp' column to datetime objects if it's not already
df_battery['timestamp'] = pd.to_datetime(df_battery['timestamp'])

# Set 'timestamp' column as the index
df_battery = df_battery.set_index('timestamp')

# Drop the 'time_utc' column
df_battery = df_battery.drop(columns=['time_utc'])

# Average the data over an hour
df_battery = df_battery.resample('H').mean()

df_battery.head()

<ipython-input-42-8e0c11ea4bdd>:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_battery = df_battery.resample('H').mean()


,total_charging,total_discharging,net_output
timestamp,,,
2024-05-31 19:00:00-05:00,-64.765250,254.316083,189.550750
2024-05-31 20:00:00-05:00,-61.557000,435.636333,374.079500
2024-05-31 21:00:00-05:00,-156.169917,198.554750,42.385167
2024-05-31 22:00:00-05:00,-296.802000,130.376417,-166.425333
2024-05-31 23:00:00-05:00,-204.772750,50.188167,-154.584667


In [ ]:
# prompt: Using dataframe df_battery: merge df_battery with df; for df_battery, using the time stored in the index, for df, use the timestamp column, drop column 'hour' from the merged df, set 'timestamp' to index
# Merge the two dataframes based on the timestamp
merged_df = pd.merge(df_battery, df, left_on='timestamp', right_on='timestamp', how='inner')

# Set the 'timestamp' column as the index
merged_df = merged_df.set_index('timestamp')
merged_df = merged_df.rename({'load.load': 'load'},axis = 1)

merged_df.to_csv(f"ercot_hourly_demand_and_price_{year}.csv")

merged_df.head()

,total_charging,total_discharging,net_output,load,spp
timestamp,,,,,
2024-05-31 19:00:00-05:00,-64.765250,254.316083,189.550750,61118.666667,57.38
2024-05-31 20:00:00-05:00,-61.557000,435.636333,374.079500,59154.000000,69.28
2024-05-31 21:00:00-05:00,-156.169917,198.554750,42.385167,57342.083333,52.55
2024-05-31 22:00:00-05:00,-296.802000,130.376417,-166.425333,54805.250000,30.50
2024-05-31 23:00:00-05:00,-204.772750,50.188167,-154.584667,51613.250000,25.45
